In [2]:
import pandas as pd
import numpy as np
import networkx as nx

In [3]:
#step1.读取上市公司和子公司数据，并进行连接
df_com = pd.read_excel(r'../../file/subject/chapter3/SSGScompany04.xlsx')
df_son = pd.read_excel(r'../../file/subject/chapter3/company_info_all05.xlsx')
df = pd.merge(df_son,df_com,left_on='上市公司名称',right_on='公司名称2').reset_index()
#step2.筛选正常营业公司
df['公司状态'] = df['公司状态'].apply(lambda x:'开业' if x == 0 else x)
df = df[-df['公司状态'].str.startswith('吊销')]
#step3.筛选非同城公司以建立关联
city_net = df[['city_x','city_y']].rename(columns={'city_x':'ZGS_city','city_y':'SSGS_city'})
city_net['count'] = 1
city_net = city_net[-(city_net['ZGS_city'] == city_net['SSGS_city'])]
#step4.处理错误城市数据
city_net = city_net[-(city_net.ZGS_city.isin([0,'','']) | city_net.ZGS_city.isna())]
city_net.ZGS_city = city_net.ZGS_city.apply(lambda x:np.nan if x.isascii() else x)
city_net.dropna(inplace=True)
#step5.聚合城市数据，统计连接次数
city_net = city_net.groupby(by=['SSGS_city','ZGS_city']).sum().reset_index()
city_net_all = city_net.sort_values(by='count',ascending=False)
#step6.生成GA数据
GA = nx.from_pandas_edgelist(city_net_all,source='SSGS_city',target='ZGS_city',edge_attr='count')
print('Done')

Done


#### 计算平均最短路径
average_shortest_path_length(G, weight=None, method=None)
https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.shortest_paths.generic.average_shortest_path_length.html?highlight=shortest#networkx-algorithms-shortest-paths-generic-average-shortest-path-length

In [4]:
nx.average_shortest_path_length(GA)

1.960957217605417

#### 计算所有城市最短路径
all_pairs_shortest_path_length(G, cutoff=None)
https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.shortest_paths.unweighted.all_pairs_shortest_path_length.html?highlight=all%20pairs%20shortest%20path%20length#networkx-algorithms-shortest-paths-unweighted-all-pairs-shortest-path-length

In [17]:
#获取整个路径的长度对象
length = nx.all_pairs_shortest_path_length(GA)

In [ ]:
#将路径长度分别加载到列表中
lst = []
for i in length:
    for k,v in i[1].items():
        ls = []
        ls.append(k)
        ls.append(v)
        lst.append(ls)     

In [23]:
#将这个二维数值丢入DataFrame对象中
df = pd.DataFrame(lst,columns=['city','path'])
df.head()

,city,path
0,深圳市,0
1,北京市,1
2,上海市,1
3,广州市,1
4,苏州市,1


In [24]:
len(df)

129960

In [32]:
#筛选出非同城既path=0的个数
len(df[df['path'] != 0])

129600

In [33]:
#筛选出城市直连path=1的个数
len(df[df['path'] == 1])

12872

In [34]:
#计算城市直接联系百分比
len(df[df['path'] == 1]) / len(df[df['path'] != 0])

0.09932098765432099

In [37]:
#计算城市直接联系和跨一步联系百分比
len(df[(df['path'] == 1) | (df['path'] == 2)]) / len(df[df['path'] != 0])

0.9373456790123457

#### 计算平均聚类系数
average_clustering(G, nodes=None, mode='dot')
https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.bipartite.cluster.average_clustering.html?highlight=cluster#networkx-algorithms-bipartite-cluster-average-clustering

In [40]:
nx.average_clustering(GA)

0.5971283696065839

#### 每个城市聚类系数

In [ ]:
nx.clustering(GA)